# 1. Implied Volatility

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from scipy.stats import norm
from dateutil import relativedelta
from warnings import filterwarnings
filterwarnings("ignore")

def imp_vol_cal (cp_flag, option_price, S0, K, T, r, q=0):
    
    log = np.log
    sqrt = np.sqrt
    exp = np.exp
    n = norm.pdf
    N = norm.cdf

    def find_price(cp_flag, S0, K, T, r, vol, q=0):
        
        d1 = (log(S0/K) + (r - q + (vol**2)/2) * T) / (vol * sqrt(T))
        d2 = d1 - vol*sqrt(T)

        if cp_flag == 'c':
            price = S0 * N(d1) - K*exp(-r*T)*N(d2)
        
        else:
            price = K * exp(-r*T)*N(-d2) - S0*exp(-q*T)*N(-d1)

        return price

    def vega_func (cp_flag, S0, K, T, r, vol, q=0):

        d1 = (log(S0/K) + (r + (vol**2)/2) * T) / (vol * sqrt(T))

        return S0 * sqrt(T) * n(d1)

    '''use above functions with Newton_Raphson_method'''

    MAX_ITERATIONS = 100
    ERROR = 10e-6

    VOL = 0.5 #Initial Volatility

    for NUM_ITERATIONS in range(0, MAX_ITERATIONS):
        
        price = find_price(cp_flag, S0, K, T, r, VOL, q=0)
        vega = vega_func(cp_flag, S0, K, T, r, VOL, q=0)

        diff = option_price - price

        if abs(diff) > ERROR:
            VOL = VOL + diff/vega  #price? diff?

        else:
            return print(f'IV = {round(VOL, 4) * 100}%', f'Iteration Num = {NUM_ITERATIONS}', f'Error = {diff}\n', sep='\n')

    return print(f'** Calculation Failed (result below shows the best answer) **', f'IV = {VOL}', f'Iteration Num = {NUM_ITERATIONS}', f'Error = {diff}\n', sep='\n')

        

In [3]:
S0 = 273.55
K = 257.5
T = 10/360
r = 0.0155

imp_vol_cal('c', 16.15, S0, K, T, r)

S0 = 273.55
K = 260
T = 10/360
r = 0.0155

imp_vol_cal('c', 14, S0, K, T, r)

IV = -inf%
Iteration Num = 15
Error = nan

IV = 21.66%
Iteration Num = 4
Error = -2.7219121534471924e-06



위에서 베가($\frac{\partial V}{\partial c}$)의 값이 0으로 가기 때문에 IV가 inf로 가게된다

# 2. Thomas Algorithm

In [4]:
def thomas(a,b,c,d):
    """ A is the tridiagnonal coefficient matrix and d is the RHS matrix"""
    N = len(a)
    cp = np.zeros(N,dtype='float64') 
    dp = np.zeros(N,dtype='float64') 
    X = np.zeros(N,dtype='float64') 
    
    cp[0] = c[0]/b[0]  
    dp[0] = d[0]/b[0]

    for i in np.arange(1,(N),1):
        dnum = b[i] - a[i]*cp[i-1]
        cp[i] = c[i]/dnum
        dp[i] = (d[i]-a[i]*dp[i-1])/dnum
    
    X[(N-1)] = dp[N-1] 

    for i in np.arange((N-2),-1,-1): 
        X[i] = (dp[i]) - (cp[i])*(X[i+1])
    
    return print('The Values of x are:', np.round(X, 3), sep=' ')

In [5]:
a = [0, -2, 0.5, -2, 1, 1]
b = [1, 1, 1, 1, 0.2, 1]
c = [2, 1, 0.5, 0.5, 0.3, 0]
d = [2, -1, 0.5, 2.3, -5, 0.25]

thomas(a, b, c, d)

The Values of x are: [  1.596   0.202   1.99   -3.182  18.926 -18.676]


정답

$$\begin{pmatrix}1.596& 0.202&1.990&-3.182&18.926&-18.676 \end{pmatrix}$$